In [1]:
# === SEKCJA 1: IMPORT I WCZYTANIE DANYCH ===
import pandas as pd
import numpy as np
import re
from IPython.display import display
import pickle
import os

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Concatenate, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

print("--- Wczytywanie danych ---")
try:
    df_main_raw = pd.read_csv('saleflats_mazowieckie_c.csv', sep=',', header=None, on_bad_lines='skip', low_memory=False)
    # Wczytujemy nasz nowy, zweryfikowany słownik
    df_slownik = pd.read_csv('slownik_finalny_z_hierarchia.csv', sep=';')
    print("Pliki wczytane pomyślnie.")
except FileNotFoundError as e:
    print(f"BŁĄD: Nie znaleziono pliku: {e.filename}.")
    raise

C:\Users\Ai\AppData\Local\miniconda3\envs\projekt_stan\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


--- Wczytywanie danych ---
Pliki wczytane pomyślnie.


In [2]:
# === SEKCJA 2: PRZYGOTOWANIE DANYCH -- Z WZBOGACANIEM OPISU ===
print("--- Przygotowanie danych do modelu ---")

# --- 2.1: Przygotowanie danych z ofert ---
df_main = df_main_raw.copy()
df_main.columns = [i for i in range(53)] + ['WojewodztwoID', 'PowiatID', 'GminaID', 'RodzajGminyID', 'MiastoID', 'DzielnicaID', 'UlicaID']
main_cols_map = {0: 'SaleId', 4: 'Description', 5: 'Area', 6: 'Price', 17: 'NumberOfRooms', 35: 'Floor', 36: 'Floors'}
df_main.rename(columns=main_cols_map, inplace=True)

numeric_features = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors']
id_features = ['UlicaID']
for col in numeric_features + id_features:
    df_main[col] = pd.to_numeric(df_main[col], errors='coerce')
df_main.dropna(subset=['Description'] + numeric_features + id_features, inplace=True)
df_main['UlicaID'] = df_main['UlicaID'].astype(int)

# --- 2.2: Łączenie ofert ze słownikiem ---
print("\nŁączenie ofert z danymi ze słownika...")
df_merged = pd.merge(df_main, df_slownik, on='UlicaID', how='inner')
print(f"Liczba ofert po połączeniu ze słownikiem: {len(df_merged)}")

if len(df_merged) == 0:
    raise ValueError("Połączenie danych nie dało żadnych wyników.")

df_model_ready = df_merged.copy()
print(f"Finalny zbiór danych gotowy. Wiersze: {len(df_model_ready)}")


# --- 2.3: Przygotowanie Danych Wejściowych (X) ---
# ==============================================================================
# === KLUCZOWA ZMIANA: WZBOGACANIE OPISU PRZED TOKENIZACJĄ ===
# ==============================================================================
print("\nWzbogacanie opisów o nazwy lokalizacji w celu wzmocnienia sygnału...")
df_model_ready['description_enriched'] = df_model_ready['Description'] + " " + df_model_ready['Dzielnica_Name'] + " " + df_model_ready['Ulica_Name']

def clean_text(text): return re.sub(r'[^a-ząęółśżźćń ]', '', str(text).lower())

# Używamy teraz nowej, wzbogaconej kolumny do nauki tokenizera
df_model_ready['description_clean'] = df_model_ready['description_enriched'].apply(clean_text)

# ==============================================================================

MAX_WORDS, MAX_LEN = 20000, 250
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<unk>")
tokenizer.fit_on_texts(df_model_ready['description_clean'])
X_text = pad_sequences(tokenizer.texts_to_sequences(df_model_ready['description_clean']), maxlen=MAX_LEN)

df_model_ready['Price_per_sqm'] = df_model_ready['Price'] / df_model_ready['Area']
df_model_ready['Price_per_sqm'].replace([np.inf, -np.inf], np.nan, inplace=True)
numeric_features_cols = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors', 'Price_per_sqm']
numeric_pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
X_numeric = numeric_pipeline.fit_transform(df_model_ready[numeric_features_cols])

# --- 2.4: Przygotowanie Danych Wyjściowych (y) ---
# ... (reszta komórki bez zmian) ...
le_dzielnica = LabelEncoder()
y_dzielnica = le_dzielnica.fit_transform(df_model_ready['Dzielnica_Name'])
num_classes_dzielnica = len(le_dzielnica.classes_)
le_ulica = LabelEncoder()
y_ulica = le_ulica.fit_transform(df_model_ready['Ulica_Name'])
num_classes_ulica = len(le_ulica.classes_)
print(f"\nProblem przygotowany do modelowania:")
print(f" - Liczba klas (dzielnice): {num_classes_dzielnica} -> {le_dzielnica.classes_[:5]}...")
print(f" - Liczba klas (ulice): {num_classes_ulica}")
train_indices, val_indices = train_test_split(range(len(df_model_ready)), test_size=0.2, random_state=42, stratify=y_dzielnica)
X_train_text, X_val_text = X_text[train_indices], X_text[val_indices]
X_train_num, X_val_num = X_numeric[train_indices], X_numeric[val_indices]
y_train_dzielnica, y_val_dzielnica = y_dzielnica[train_indices], y_dzielnica[val_indices]
y_train_ulica, y_val_ulica = y_ulica[train_indices], y_ulica[val_indices]
print("\nDane podzielone na zbiory treningowe i walidacyjne.")

--- Przygotowanie danych do modelu ---

Łączenie ofert z danymi ze słownika...
Liczba ofert po połączeniu ze słownikiem: 9548
Finalny zbiór danych gotowy. Wiersze: 9548

Wzbogacanie opisów o nazwy lokalizacji w celu wzmocnienia sygnału...

Problem przygotowany do modelowania:
 - Liczba klas (dzielnice): 18 -> ['Bemowo' 'Białołęka' 'Bielany' 'Mokotów' 'Ochota']...
 - Liczba klas (ulice): 703

Dane podzielone na zbiory treningowe i walidacyjne.


In [3]:
# === SEKCJA 3: BUDOWA I TRENING MODELU HIERARCHICZNEGO ===

# --- 3.1: Definicja architektury ---
# Wejścia
input_text = Input(shape=(MAX_LEN,), name='text_input')
input_numeric = Input(shape=(X_numeric.shape[1],), name='numeric_input')

# Wspólny trzon
text_embedding = Embedding(input_dim=MAX_WORDS, output_dim=128)(input_text)
lstm_out = LSTM(128, dropout=0.3)(text_embedding)
concatenated = Concatenate()([lstm_out, input_numeric])
common_dense = Dense(128, activation='relu')(concatenated)
common_dense = Dropout(0.5)(common_dense)

# Gałąź wyjściowa dla DZIELNICY
dzielnica_branch = Dense(64, activation='relu')(common_dense)
dzielnica_output = Dense(num_classes_dzielnica, activation='softmax', name='output_dzielnica')(dzielnica_branch)

# Gałąź wyjściowa dla ULICY
ulica_branch = Dense(256, activation='relu')(common_dense)
ulica_output = Dense(num_classes_ulica, activation='softmax', name='output_ulica')(ulica_branch)

# --- 3.2: Kompilacja modelu ---
model = Model(inputs=[input_text, input_numeric], outputs=[dzielnica_output, ulica_output])

# Definiujemy osobne straty dla każdego wyjścia
losses = {
    "output_dzielnica": "sparse_categorical_crossentropy",
    "output_ulica": "sparse_categorical_crossentropy",
}

# Definiujemy wagi dla każdej ze strat
loss_weights = {
    "output_dzielnica": 1.0,
    "output_ulica": 0.5
}

# POPRAWKA: Definiujemy metryki dla każdego wyjścia osobno
metrics = {
    "output_dzielnica": "accuracy",
    "output_ulica": "accuracy"
}

model.compile(
    optimizer='adam',
    loss=losses,
    loss_weights=loss_weights,
    metrics=metrics  # Przekazujemy słownik metryk
)
model.summary()

# --- 3.3: Trening ---
X_train = [X_train_text, X_train_num]
y_train = {'output_dzielnica': y_train_dzielnica, 'output_ulica': y_train_ulica}
X_val = [X_val_text, X_val_num]
y_val = {'output_dzielnica': y_val_dzielnica, 'output_ulica': y_val_ulica}

callbacks = [
    EarlyStopping(
        monitor='val_output_dzielnica_accuracy', 
        patience=5, 
        restore_best_weights=True, 
        verbose=1,
        mode='max'  # <-- DODAJ TĘ LINIĘ
    ),
    ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1)
]

print("\nRozpoczynam trening modelu hierarchicznego...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=128,
    callbacks=callbacks
)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)       │ (None, 250)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 250, 128)          │       2,560,000 │ text_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ (None, 128)               │         131,584 │ embedding[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ numeric_input (InputLayer)    │ (None, 6)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 134)               │               0 │ lstm[0][0],                │
│                               │                           │                 │ numeric_input[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 128)               │          17,280 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 128)               │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 64)                │           8,256 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 256)               │          33,024 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_dzielnica (Dense)      │ (None, 18)                │           1,170 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_ulica (Dense)          │ (None, 703)               │         180,671 │ dense_2[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,931,985 (11.18 MB)

 Trainable params: 2,931,985 (11.18 MB)

 Non-trainable params: 0 (0.00 B)


Rozpoczynam trening modelu hierarchicznego...
Epoch 1/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 27s 393ms/step - loss: 5.2801 - output_dzielnica_accuracy: 0.2935 - output_dzielnica_loss: 2.4114 - output_ulica_accuracy: 0.1368 - output_ulica_loss: 5.7370 - val_loss: 3.2294 - val_output_dzielnica_accuracy: 0.5869 - val_output_dzielnica_loss: 1.1693 - val_output_ulica_accuracy: 0.2157 - val_output_ulica_loss: 4.1204 - learning_rate: 0.0010
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 22s 366ms/step - loss: 2.9545 - output_dzielnica_accuracy: 0.6910 - output_dzielnica_loss: 0.9790 - output_ulica_accuracy: 0.2368 - output_ulica_loss: 3.9509 - val_loss: 2.1877 - val_output_dzielnica_accuracy: 0.8445 - val_output_dzielnica_loss: 0.5166 - val_output_ulica_accuracy: 0.3068 - val_output_ulica_loss: 3.3420 - learning_rate: 0.0010
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 22s 362ms/step - loss: 2.1116 - output_dzielnica_accuracy: 0.8504 - output_dzielnica_loss: 0.5039 - output_ulica_accuracy: 0.3276 - output_ulica_loss

In [4]:
# === SEKCJA 4: ZAPIS ARTEFAKTÓW DO PRODUKCJI ===
artifacts_dir = 'model_artifacts_final'
os.makedirs(artifacts_dir, exist_ok=True)

# 1. Zapis modelu
model.save(os.path.join(artifacts_dir, 'final_hierarchical_model.keras'))

# 2. Zapis Tokenizera
with open(os.path.join(artifacts_dir, 'tokenizer.pkl'), 'wb') as f: pickle.dump(tokenizer, f)

# 3. Zapis pipelinu numerycznego
with open(os.path.join(artifacts_dir, 'numeric_pipeline.pkl'), 'wb') as f: pickle.dump(numeric_pipeline, f)

# 4. Zapis koderów dla zmiennych celu
with open(os.path.join(artifacts_dir, 'le_dzielnica.pkl'), 'wb') as f: pickle.dump(le_dzielnica, f)
with open(os.path.join(artifacts_dir, 'le_ulica.pkl'), 'wb') as f: pickle.dump(le_ulica, f)

print(f"\nWszystkie artefakty zostały zapisane w folderze: '{artifacts_dir}'")


Wszystkie artefakty zostały zapisane w folderze: 'model_artifacts_final'
